## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference.
    * Extract out professors who do not teach
    * Professors who do teach work at the maximum of their workload
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen

In [ ]:
import pandas as pd
from docplex.mp.model import Model
import math
import random
import numpy as np
import itertools

### Functions and Customizations

In [ ]:
## Standardize data frame column names
def clean_dataframe_columns(df:pd.DataFrame) -> pd.DataFrame:
    colnames = df.columns.values.tolist()
    colnames = [col.replace(' ', '_') for col in colnames]
    df.columns = colnames

In [ ]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    ''''''
    def get_day_name(day:str):
        days_name = {'M':'Monday', 'TU':'Tuesday', 'W':'Wednesday', 'TH':'Thursday', 'F':'Friday'}
        return days_name[day]
    
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Session'})
    result = result.sort_values(by=['Course_Number', 'Session']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department_Code', 'Course_Number', 'Course_Name'])
    
    courses_offered = result
    result = pd.DataFrame()
    
    for course in courses_offered.itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        days_taught = course['Day'][0]
        days_taught = days_taught.split('/')
        for i in days_taught:
            course['Day'] = i
            result = result.append([course], ignore_index=True)
            
    result['Day'] = result.apply(lambda row : get_day_name(row.Day), axis=1)
    return result

In [ ]:
## Clean
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload_Credit_Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty_Name', 'Department_Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

In [ ]:
def get_period_df(start_hour=8, end_hour=18) -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    days_of_weeks = dict(zip(days, range(0, 5)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 101)), 'Day_Name' : day_names, 'Day_Number' : day_digits, 
                                   'Start_Time' : start_time, 'End_Time' : end_time})


In [ ]:
def get_column_value_as_list(df:pd.DataFrame, colname:str) -> pd.DataFrame:
    '''Get the values of a data frame column as a Python list'''
    if colname in df.columns:
        return df[colname].values.tolist()
    else:
        return None

### Data Exploration and Cleaning

In [ ]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)
print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
clean_dataframe_columns(classrooms_df)
display(file_sheets[0], classrooms_df)

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
clean_dataframe_columns(colleges_and_depts)
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
clean_dataframe_columns(course_catalog_df)
display(file_sheets[2], course_catalog_df.sort_values('Course_Name').head(25))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
clean_dataframe_columns(courses_offered)
display(file_sheets[3], courses_offered.sort_values('Course_Name').head(45))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
clean_dataframe_columns(professors_df)
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
clean_dataframe_columns(qualifications_df)
display(file_sheets[5], qualifications_df.head(5))

In [ ]:
# For reproducibility
random.seed(3)

courses_offered = standardize_courses_offered(courses_offered, course_catalog_df)
preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df)
periods_df = get_period_df()
display(periods_df.head(21))

### Mathematical Model

#### Model Initialization

In [ ]:
## Model Environment
model = Model(name='Course Scheduling')

######################
# Decision Variables #
######################
x_fcsrp = dict()
for professor in preferences_df.Faculty_Name:
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] = model.binary_var(name = "x({},{},{},{},{})".format(professor, course.Course_Name, course.Session, room, period))
                
model.print_information()

#### Constraints

In [ ]:
### Exclude faculty member whose workload is zero
for professor in preferences_df[preferences_df.Workload_Credit_Hours == 0].itertuples():
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor.Faculty_Name, course.Course_Name, course.Session, room, period)] == 0)

### Each course is taught for the number of periods needed to satisfy the course credit hours on the days it is offered
CLASS_DURATION = 50
PERIOD_DURATION = 30
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    number_of_credit = course[1].Credit_Hours.values.tolist()[0]
    duration = (number_of_credit * CLASS_DURATION) / (len(course[1]) * PERIOD_DURATION)
    number_of_period_covered = math.ceil(duration)
    for day_offered in course[1].Day.values.tolist():
        aggregate_values = []
        for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, period)])
        model.add_constraint(model.sum(aggregate_values) == number_of_period_covered)
        
### A faculty can only teach one course at a time
for professor in preferences_df.Faculty_Name:
    for days in courses_offered.groupby(by='Day'):
        for period in periods_df[periods_df.Day_Name == days[0]].Period.values.tolist():
            aggregate_values = []
            for course in days[1].itertuples():
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)])
            model.add_constraint(model.sum(aggregate_values) <= 1)               

### Only one room can host a class in any given time
for room in classrooms_df.Classroom:
    for days in courses_offered.groupby(by='Day'):
        for period in periods_df[periods_df.Day_Name == days[0]].Period.values.tolist():
            aggregate_values = []
            for course in days[1].itertuples():
                for professor in preferences_df.Faculty_Name:
                    aggregate_values.append(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)])
            model.add_constraint(model.sum(aggregate_values) <= 1)               

### The same room is assigned one class
CLASS_DURATION = 50
PERIOD_DURATION = 30
for room in classrooms_df.Classroom:
    for course_df in courses_offered.groupby(by=['Course_Name', 'Session']):
        course_name, course_session = course_df[0]
        course = course_df[1]
        number_of_credit = course.Credit_Hours.values.tolist()[0]
        duration = (number_of_credit * CLASS_DURATION) / (len(course) * PERIOD_DURATION)
        number_of_period_covered = math.ceil(duration)
        for day in course.Day.values.tolist():
            for professor in preferences_df.Faculty_Name:
                aggregate_values = []
                for period in periods_df[periods_df.Day_Name == day].Period.values.tolist():
                    aggregate_values.append(x_fcsrp[(professor, course_name, course_session, room, period)])
                model.add_constraint((model.sum(aggregate_values) == 0) + (model.sum(aggregate_values) == number_of_period_covered) >= 1)

### Each course is taught at the same time in the same room by the same faculty on the day it is offered
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    days_offered = course[1].Day.values.tolist()
    for index in range(len(days_offered) - 1):
        x = periods_df[periods_df.Day_Name == days_offered[index]].Period.values.tolist()
        y = periods_df[periods_df.Day_Name == days_offered[index + 1]].Period.values.tolist()
        for periods in list(zip(x, y)):
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df['Classroom']:
                    aggregate_values = []
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[0])])
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[1])])
                    model.add_constraint(aggregate_values[0] - aggregate_values[1] == 0)

### A course is taught in a classroom that can hold at least the maximum number of students enrolled
for course in courses_offered.itertuples():
    for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.itertuples():
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room.Classroom, period)] * course.Max_Enrollement <= room.Capacity)

### Each faculty member can only teach up to his or her workload
CLASS_DURATION = 50
PERIOD_DURATION = 30
for professor in preferences_df.itertuples():
    aggregate_values = []
    for course in courses_offered.groupby(by=['Course_Name', 'Session']):
        course_name, session = course[0]
        for day_offered in course[1].Day.values.tolist():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor.Faculty_Name, course_name, session, room, period)])
    model.add_constraint(model.sum(aggregate_values) * PERIOD_DURATION <= professor.Workload_Credit_Hours * 60)

#### Special Rules

1. Seniors courses start at 1:00 PM

In [ ]:
### 800+ courses are taught after 1:00 PM
SENIOR_COURSE_NUMBER = 800
START_TIME = 13 # PM
for course in courses_offered[courses_offered.Course_Number >= SENIOR_COURSE_NUMBER].itertuples():
    for period in periods_df[(periods_df.Day_Name == course.Day) & (periods_df.Start_Time < START_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)


2. No classes are taught on Friday after 4 PM

In [ ]:
HOLIDAY = 'Friday'
END_TIME = 16
for course in courses_offered[courses_offered.Day == HOLIDAY].itertuples():
    for period in periods_df[(periods_df.Day_Name == HOLIDAY) & (periods_df.Start_Time >= END_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)

3. Classes are taught on consecutive periods

#### Objective Function

In [ ]:
total_preference = 0
for professor in preferences_df.itertuples():
    professor_df = pd.DataFrame(professor).T.drop(columns=0)
    professor_df.columns = preferences_df.columns
    for course in courses_offered.itertuples():
        course_code = '{}_{}'.format(course.Department_Code, str(course.Course_Number))
        preference_score = professor_df[course_code].values.tolist()[0]
        professor_name = professor_df.Faculty_Name.values.tolist()[0]
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                total_preference += preference_score * x_fcsrp[(professor_name, course.Course_Name, course.Session, room, period)]

model.maximize(total_preference)
model.print_information()

#### Solution

In [ ]:
###############
# Solve Model #
###############
model.print_information()
solution = model.solve(log_output = True)
assert solution, "Solve Failed"
model.report()

#### Post-processing

In [ ]:
########################
## View model solution #
########################
classes_assigned = []
professors_assigned = []
professors = []
courses = []
course_codes = []
rooms = []
sessions = []
periods = []
days = []
workloads = []
credits = []
room_capacity = []
course_enrollements = []
start_times = []
end_times = []

for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    course_code = course[1].Course_Number.values.tolist()[0]
    course_credit = course[1].Credit_Hours.values.tolist()[0]
    course_enrollment = course[1].Max_Enrollement.values.tolist()[0]
    for day_offered in course[1].Day.values.tolist():
        for professor in preferences_df.itertuples():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.itertuples():
                    value = model.solution.get_value(x_fcsrp[(professor.Faculty_Name, course_name, session, room.Classroom, period)])
                    if value != 0:
                        professors.append(professor.Faculty_Name)
                        courses.append(course_name)
                        course_codes.append(course_code)
                        sessions.append(session)
                        rooms.append(room.Classroom)
                        days.append(day_offered)
                        periods.append(period)
                        workloads.append(professor.Workload_Credit_Hours)
                        credits.append(course_credit)
                        room_capacity.append(room.Capacity)
                        course_enrollements.append(course_enrollment)
                        start_times.append(periods_df[periods_df.Period == period].Start_Time.values.tolist()[0])
                        end_times.append(periods_df[periods_df.Period == period].End_Time.values.tolist()[0])
                        
                        if course_name not in classes_assigned:
                            classes_assigned.append(course_name)
                        if professor_name not in professors_assigned:
                            professors_assigned.append(professor_name)
    
print(f'Number of courses assigned {len(classes_assigned)}')
print(f'Number of courses offered  {len(courses_offered.groupby(by=["Course_Name", "Session"]))}')
print("\nAll of the following courses have been assigned")
print(set(classes_assigned).difference(set(courses_offered.Course_Name.values.tolist())))
print(set(courses_offered.Course_Name.values.tolist()).difference(set(classes_assigned)))

print(set(professors_assigned).difference(set(qualifications_df.Faculty_Name.values.tolist())))

class_schedule = pd.DataFrame({'Professor' : professors, 'Workload' : workloads, 'Course_Name': courses, 
                               'Course_Code' : course_codes, 'Credit' : credits, 'Session' : sessions, 
                               'Period' : periods, 'From' : start_times, 'To': end_times, 'Day_Offered' : days, 
                               'Max_Enrollement' : course_enrollements, 'Classroom' : rooms, 'Room Capacity' : room_capacity})

class_schedule.sort_values(by=['Course_Name', 'Session', 'Day_Offered', 'Period'])
for course in class_schedule.groupby(by=['Professor']):
    display(course[1])
    
### Write output
class_schedule.to_excel(excel_writer='Classes Schedule_Consecutive Periods.xlsx', sheet_name='Report')